**Convolutional Neural Networks - CNN**

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/flavio-mota/si-rna-ag-2025/blob/main/CNN/Aula_8_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

### Configurando o ambiente e bibliotecas

Nesta primeira parte do notebook, vamos carregar as bibliotecas que usaremos ao longo da aula. Elas são responsáveis por tarefas como:

- Carregar e manipular o conjunto de dados de imagens;
- Construir a arquitetura da nossa rede neural convolucional (CNN);
- Treinar o modelo e avaliar o desempenho;
- Utilizar modelos pré-treinados em bases grandes como o ImageNet.

A ideia é que, a partir deste ponto, tudo o que fizermos em termos de CNN seja construído sobre essas ferramentas. Não vamos nos aprofundar em cada função individual, mas é importante ter em mente que o `TensorFlow/Keras` é o “motor” por trás da criação e treinamento dos modelos que veremos hoje.

In [ ]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [ ]:
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

Vamos definir os tamanhos de fonte padrão para deixar as figuras mais bonitas:

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

Estes códigos podem ser muito lentos sem uma GPU, então vamos garantir que haja uma, ou então emitir um aviso:

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if not tf.config.list_physical_devices('GPU'):
    print("Nenhuma GPU foi detectada. Redes neurais podem ser muito lentas sem uma GPU.")
    if IS_COLAB:
        print("Acesse Runtime > Alterar runtime e selecione um acelerador de hardware de GPU.")

# Arquiteturas de CNN

### O dataset Fashion-MNIST

Para estudar redes convolucionais, vamos utilizar o conjunto de dados **Fashion-MNIST**. Ele é uma evolução do clássico MNIST: em vez de dígitos escritos à mão, temos imagens de peças de roupa (camisetas, calçados, bolsas etc.).

Algumas características importantes:

- As imagens são em escala de cinza, com tamanho 28×28 pixels;
- Cada imagem pertence a uma de 10 classes diferentes;
- Possui 60.000 imagens para treino e 10.000 para teste.

Esse dataset é interessante porque é simples o suficiente para rodar em sala de aula, mas já é mais desafiador do que o MNIST original, o que nos permite ver melhor a vantagem de usar CNNs em relação a redes totalmente conectadas (MLPs).

### Pré-processamento: por que normalizar as imagens?

Antes de enviar as imagens para a rede, precisamos fazer um pequeno pré-processamento. Os valores dos pixels no Fashion-MNIST vão de 0 a 255. Quando dividimos tudo por 255, trazemos esses valores para o intervalo [0, 1].

Essa normalização ajuda a rede de várias maneiras:

- Deixa a escala dos dados mais estável para o processo de otimização;
- Facilita o aprendizado dos pesos, já que os gradientes tendem a ficar em uma faixa mais “controlada”;
- Costuma acelerar a convergência do treinamento.

Além disso, aqui também ajustamos o formato das imagens (por exemplo, adicionando o canal “1” para indicar que são imagens em tons de cinza), de forma que a CNN consiga interpretar corretamente a estrutura dos dados.


In [ ]:
# código extra – carrega o conjunto de dados MNIST, adiciona o eixo dos canais às entradas,
# dimensiona os valores para o intervalo de 0 a 1 e divide o conjunto de dados
mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_train_full = np.expand_dims(X_train_full, axis=-1).astype(np.float32) / 255
X_test = np.expand_dims(X_test.astype(np.float32), axis=-1) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

### Construindo a arquitetura da CNN

Agora vamos, de fato, definir a arquitetura da nossa **Convolutional Neural Network**. A grande diferença em relação a uma MLP é que aqui aproveitamos a estrutura espacial da imagem: ao invés de “achatar” tudo logo no início, usamos camadas que varrem a imagem com filtros (kernels) para extrair padrões locais.

A arquitetura que vamos usar segue uma lógica bem comum em redes convolucionais simples:

- Camadas **convolucionais**: aplicam filtros que detectam padrões como bordas, texturas e pequenos detalhes;
- Camadas de **pooling** (como MaxPooling): reduzem a resolução espacial, mantendo as características mais importantes e diminuindo o número de parâmetros;
- Camada **Flatten**: transforma o mapa de características em um vetor;
- Camadas **densas** ao final: combinam essas características extraídas para realizar a classificação em uma das 10 classes.

Essa é uma CNN relativamente pequena, mas já é suficiente para mostrar, na prática, como convoluções melhoram o desempenho em tarefas de visão computacional.

In [ ]:
from functools import partial

tf.random.set_seed(42)
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same",
                        activation="relu", kernel_initializer="he_normal")
model = tf.keras.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation="softmax")
])

### Compilando e treinando a rede convolucional

Com a arquitetura definida, o próximo passo é **compilar** e **treinar** o modelo. Ao compilar, definimos três elementos principais:

- A **função de perda**: aqui usamos `sparse_categorical_crossentropy`, adequada para problemas de classificação com múltiplas classes e rótulos inteiros;
- O **otimizador**: utilizaremos o `Adam`, que é um método de otimização adaptativo muito utilizado em deep learning;
- As **métricas**: vamos acompanhar especialmente a acurácia, que indica a porcentagem de acertos na classificação.

Durante o treinamento (`model.fit`), a rede ajusta os pesos dos filtros e das camadas densas, tentando minimizar a função de perda. Também usamos um conjunto de validação para observar se o modelo está generalizando bem ou começando a sofrer overfitting.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test)
X_new = X_test[:10]  # Finge que temos novas imagens
y_pred = model.predict(X_new)

### Por que utilizar modelos pré-treinados?

Treinar redes convolucionais profundas do zero exige muito poder computacional e grandes volumes de dados. Para muitas aplicações práticas, isso é inviável ou desnecessário. Uma alternativa é aproveitar modelos que já foram treinados em bases enormes, como a ImageNet, e reutilizar esse conhecimento.

Esses modelos pré-treinados aprenderam filtros capazes de detectar desde padrões simples (bordas, texturas) até conceitos mais complexos (partes de objetos, formas específicas). Podemos:

- Utilizá-los diretamente para classificação em tarefas semelhantes àquelas de treino;
- Usá-los como extratores de características, adicionando novas camadas ao final (técnica conhecida como *transfer learning*).

Nesta parte do notebook, vamos carregar e utilizar uma rede pré-treinada famosa para observar, na prática, como ela classifica imagens reais.

### A arquitetura ResNet50

Aqui vamos carregar a **ResNet50**, uma rede convolucional profunda composta por 50 camadas treinadas no dataset ImageNet. A grande inovação da família ResNet é o uso de conexões de atalho (*skip connections*), que ajudam a evitar problemas de desaparecimento do gradiente em redes muito profundas.

Ao carregar `ResNet50(weights="imagenet")`, estamos trazendo um modelo que já “viu” mais de um milhão de imagens e aprendeu a reconhecer mil classes diferentes. Nosso objetivo nesta aula não é treinar essa rede, e sim aproveitar o modelo já treinado para fazer classificações em imagens de exemplo.

In [ ]:
model = tf.keras.applications.ResNet50(weights="imagenet")

**Aviso**: A expressão `load_sample_images()["images"]` retorna uma lista de imagens em Python. No entanto, nas versões mais recentes, o Keras não aceita mais listas em Python, então precisamos converter essa lista em um tensor. Podemos fazer isso usando `tf.constant()`, mas aqui usei `K.constant()`: ele simplesmente chama `tf.constant()` se você estiver usando o TensorFlow como backend (como é o caso aqui), mas se você decidir usar JAX ou PyTorch como backend, `K.constant()` chamará a função apropriada do backend escolhido.

### Preparando as imagens para a ResNet50

Modelos pré-treinados como a ResNet50 esperam que as imagens de entrada tenham um formato específico. No caso da ResNet50:

- As imagens precisam ter tamanho 224×224 pixels;
- Devem ter 3 canais de cor (RGB);
- Passam por um pré-processamento próprio, definido na implementação da Keras.

Nesta célula, vamos:

- Carregar algumas imagens de exemplo;
- Redimensioná-las para 224×224;
- Organizar o tensor de forma adequada para ser enviado ao modelo.

Esse cuidado com o pré-processamento é fundamental: se o formato da imagem não estiver compatível com o modelo, a rede não consegue aproveitar corretamente o conhecimento que adquiriu durante o treinamento no ImageNet.

In [ ]:
K = tf.keras.backend
images = K.constant(load_sample_images()["images"])
images_resized = tf.keras.layers.Resizing(height=224, width=224,
                                          crop_to_aspect_ratio=True)(images)

In [ ]:
inputs = tf.keras.applications.resnet50.preprocess_input(images_resized)

In [ ]:
Y_proba = model.predict(inputs)
Y_proba.shape

In [ ]:
top_K = tf.keras.applications.resnet50.decode_predictions(Y_proba, top=3)
for image_index in range(len(images)):
    print(f"Image #{image_index}")
    for class_id, name, y_proba in top_K[image_index]:
        print(f"  {class_id} - {name:12s} {y_proba:.2%}")

In [ ]:
plt.figure(figsize=(10, 6))
for idx in (0, 1):
    plt.subplot(1, 2, idx + 1)
    plt.imshow(images_resized[idx] / 255)
    plt.axis("off")

plt.show()

## Exercício prático: classificando imagens com um modelo pré-treinado

Neste exercício, você vai experimentar na prática o uso de um modelo pré-treinado para classificação de imagens.

**Tarefa:**  
Escolha **três imagens reais** (podem ser fotos do seu celular ou imagens da internet) contendo objetos bem definidos, como animais, veículos, alimentos, ferramentas ou outros itens do dia a dia. Em seguida:

1. Carregue essas imagens no notebook;
2. Faça o pré-processamento necessário (redimensionamento e normalização) de acordo com o modelo pré-treinado escolhido (por exemplo, ResNet50);
3. Use o modelo para obter as previsões;
4. Registre, para cada imagem:
   - As classes previstas e suas probabilidades;
   - Se o resultado faz sentido ou não;
   - Possíveis motivos para erros ou confusões (ângulo da foto, iluminação, objeto muito diferente do que a rede “espera” etc.).

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

model =

img_path = "sua_imagem.jpg"
img = image.load_img(img_path, target_size=(224, 224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print(decode_predictions(preds, top=3)[0])
